In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
)

movies = [
  {
    "question": "듄 파트 투 영화에 대해 설명해줘",
    "answer": """
영화이름: Dune: Part Two
감독: Denis Villeneuve
주요출연진: Timothée Chalamet, Zendaya
예산: 190,000,000 USD
흥행수익: 700,000,000 USD
장르: SF / 어드벤처
시놉시스: 폴 아트레이디스가 프레멘과 함께 황제와 하코넨 가문에 맞서며 자신의 운명과 예언을 받아들이는 과정을 그린 서사적 속편.
"""
  },
  {
    "question": "인사이드 아웃 2 영화에 대해 설명해줘",
    "answer": """
영화이름: Inside Out 2
감독: Kelsey Mann
주요출연진: Amy Poehler, Maya Hawke
예산: 200,000,000 USD
흥행수익: 1,600,000,000 USD
장르: 애니메이션 / 가족
시놉시스: 사춘기에 접어든 라일리의 마음속에 새로운 감정들이 등장하며 감정 컨트롤 본부에 혼란이 찾아온다.
"""
  },
  {
    "question": "데드풀과 울버린 영화에 대해 설명해줘",
    "answer": """
영화이름: Deadpool & Wolverine
감독: Shawn Levy
주요출연진: Ryan Reynolds, Hugh Jackman
예산: 200,000,000 USD
흥행수익: 1,300,000,000 USD
장르: 액션 / 코미디
시놉시스: 데드풀과 울버린이 멀티버스의 균열 속에서 만나 예측 불가능한 방식으로 팀업하게 되는 R등급 마블 영화.
"""
  },
  {
    "question": "고질라 X 콩 새로운 제국 영화에 대해 설명해줘",
    "answer": """
영화이름: Godzilla x Kong: The New Empire
감독: Adam Wingard
주요출연진: Rebecca Hall, Brian Tyree Henry
예산: 150,000,000 USD
흥행수익: 560,000,000 USD
장르: 액션 / SF
시놉시스: 고질라와 콩이 새로운 위협에 맞서기 위해 불안정한 동맹을 맺으며 지구의 균형을 건 전투에 돌입한다.
"""
  }
]



template = """
You are a movie expert AI.
The examples below show the required answer format.
Always follow this format exactly.

Human: {question}
AI: {answer}
"""


prompt_template = PromptTemplate.from_template(template)

prompt = FewShotPromptTemplate(
    example_prompt=prompt_template,
    examples=movies,
    suffix="Human:{movie} 영화에 대해 설명해줘",
    input_variables=["movie"]
)


chain = prompt | chat

chain.invoke({
    "movie": "주토피아2"
})
    

AIMessage(content='AI:\n영화이름: Zootopia 2\n감독: Byron Howard, Jared Bush\n주요출연진: Ginnifer Goodwin, Jason Bateman\n예산: 180,000,000 USD\n흥행수익: 1,000,000,000 USD\n장르: 애니메이션 / 어드벤처\n시놉시스: 주토피아에서 새로운 사건이 발생하며 주인공 주디와 닉이 다시 모험에 뛰어들게 되는 이야기를 그린 애니메이션 영화.')